In [1]:
import numpy as np

In [2]:
task_order = np.zeros(5030, dtype=int)

In [3]:
task_order[:1510] = 0
task_order[1510:3020] = 1
task_order[3020:] = 2

In [4]:
task_order

In [5]:
np.save('task_order.npy', task_order)